In [55]:
pip install kaggle

In [56]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [57]:
!kaggle datasets download -d kazanova/sentiment140

sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [58]:
from zipfile import ZipFile
dataset = '/content/sentiment140.zip'

with ZipFile(dataset,'r') as f:
  f.extractall()
  print('extracted')

extracted


In [59]:
import pandas as pd
import numpy as np

In [60]:
columns = ['target','id','date','flag','user','text']
df = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding ='latin1',names=columns)

In [61]:
df.head(2)

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...


In [62]:
df.shape

(1600000, 6)

In [63]:
df.isna().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [64]:
df = df[['target','text']]

In [65]:
df['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [66]:
df.replace({'target':{4:1}},inplace=True)

<ipython-input-66-3fc55646869b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.replace({'target':{4:1}},inplace=True)


In [67]:
df['text'].apply(lambda x: True if x.startswith('@') else False).value_counts()

False    901921
True     698079
Name: text, dtype: int64

In [68]:
df['text'] = df['text'].str.replace("@[\w]*", '')

<ipython-input-68-2ba871f0330c>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace("@[\w]*", '')


In [69]:
def remove_http(text):
    if text.startswith('http') or text.startswith(' http'):
        parts = text.split('-')
        return parts[1] if len(parts)>1 else ''
    else:
        return text

In [70]:
df['text'] = df['text'].apply(remove_http)

In [71]:
df['text'] = df['text'].apply(lambda x: x.lstrip())

In [72]:
blanks = []
for i,target,text in df.itertuples():
    if type(text)==str and text.isspace() == True:
        blanks.append(i)
    elif len(text)<3:
        blanks.append(i)
len(blanks)

6736

In [73]:
df.drop(blanks,inplace = True)

In [74]:
import nltk
nltk.download('stopwords')

from nltk.stem import PorterStemmer
p_stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [75]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [76]:
def stemming(content):
    content = content.lower().split()
    stem_content = [p_stemmer.stem(word) for word in content if word not in stop_words]
    stem_content = ' '.join(stem_content)
    return stem_content

In [77]:
df['new_text'] = df['text'].apply(stemming)

In [78]:
df.head()

,target,text,new_text
0,0,"Awww, that's a bummer. You shoulda got David ...","awww, that' bummer. shoulda got david carr thi..."
1,0,is upset that he can't update his Facebook by ...,upset can't updat facebook text it... might cr...
2,0,I dived many times for the ball. Managed to sa...,dive mani time ball. manag save 50% rest go bound
3,0,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,"no, it's not behaving at all. i'm mad. why am ...","no, behav all. i'm mad. here? can't see there."


In [79]:
x = df['new_text']
y = df['target']

In [80]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB

from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

In [81]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.1,random_state=42)

In [82]:
text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',MultinomialNB())])

In [ ]:
text_clf.fit(x_train,y_train)

In [ ]:
predictions = text_clf.predict(x_test)

In [ ]:
confusion_matrix(y_test,predictions)

array([[61560, 17826],
       [20281, 59660]])

In [ ]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.75      0.78      0.76     79386
           1       0.77      0.75      0.76     79941

    accuracy                           0.76    159327
   macro avg       0.76      0.76      0.76    159327
weighted avg       0.76      0.76      0.76    159327



In [ ]:
print(accuracy_score(y_test,predictions))

0.7608252210861938
